<a href="https://colab.research.google.com/github/Saladdine-MW/tp-rag-student-version/blob/main/TP_RAG_Saladdine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/Saladdine-MW/tp-rag-student-version.git

Cloning into 'tp-rag-student-version'...
remote: Enumerating objects: 75, done.
remote: Total 75 (delta 0), reused 0 (delta 0), pack-reused 75 (from 1)
Receiving objects: 100% (75/75), 93.16 MiB | 36.12 MiB/s, done.
